In [21]:
from great_expectations.profile.user_configurable_profiler import UserConfigurableProfiler
%matplotlib inline

from great_expectations.core.expectation_configuration import ExpectationConfiguration
from great_expectations.data_context.types.resource_identifiers import (
    ExpectationSuiteIdentifier,
)

import great_expectations as ge
from great_expectations.cli.datasource import sanitize_yaml_and_save_datasource, check_if_datasource_name_exists
from ruamel.yaml import YAML

yaml=YAML()

#1. Get project context

If the notebook is not in the root dir of the great_expectations dir, you need to give the path of GE root dir.

In [2]:
ge_project_root_dir="../data_validation/GE/great_expectations"
data_context = ge.data_context.DataContext(context_root_dir=ge_project_root_dir)

#2 Create a data source

In [3]:
# you can name it as you want
datasource_name = "french_income"

my_datasource_yaml = f"""
name: {datasource_name}
class_name: Datasource
execution_engine:
  class_name: PandasExecutionEngine
data_connectors:
  default_inferred_data_connector_name:
    class_name: InferredAssetFilesystemDataConnector
    base_directory: /home/pliu/data_set/insee_income
    default_regex:
      group_names:
        - data_asset_name
      pattern: (.*)
  default_runtime_data_connector_name:
    class_name: RuntimeDataConnector
    batch_identifiers:
      - default_identifier_name
"""
print(my_datasource_yaml)


name: french_income
class_name: Datasource
execution_engine:
  class_name: PandasExecutionEngine
data_connectors:
  default_inferred_data_connector_name:
    class_name: InferredAssetFilesystemDataConnector
    base_directory: /home/pliu/data_set/insee_income
    default_regex:
      group_names:
        - data_asset_name
      pattern: (.*)
  default_runtime_data_connector_name:
    class_name: RuntimeDataConnector
    batch_identifiers:
      - default_identifier_name



In [4]:
data_context.test_yaml_config(yaml_config=my_datasource_yaml)

Attempting to instantiate class from config...
	Instantiating as a Datasource, since class_name is Datasource
	Successfully instantiated Datasource


ExecutionEngine class name: PandasExecutionEngine
Data Connectors:
	default_inferred_data_connector_name : InferredAssetFilesystemDataConnector

	Available data_asset_names (3 of 4):
		base_etablissement_par_tranche_effectif.csv (1 of 1): ['base_etablissement_par_tranche_effectif.csv']
		name_geographic_information.csv (1 of 1): ['name_geographic_information.csv']
		net_salary_per_town_categories.csv (1 of 1): ['net_salary_per_town_categories.csv']

	Unmatched data_references (0 of 0):[]

	default_runtime_data_connector_name:RuntimeDataConnector

	Available data_asset_names (0 of 0):
		Note : RuntimeDataConnector will not have data_asset_names until they are passed in through RuntimeBatchRequest

	Unmatched data_references (0 of 0): []



In [6]:
sanitize_yaml_and_save_datasource(data_context, my_datasource_yaml, overwrite_existing=False)

In [7]:
data_context.list_datasources()

[{'execution_engine': {'module_name': 'great_expectations.execution_engine',
   'class_name': 'PandasExecutionEngine'},
  'class_name': 'Datasource',
  'module_name': 'great_expectations.datasource',
  'data_connectors': {'default_inferred_data_connector_name': {'default_regex': {'group_names': ['data_asset_name'],
     'pattern': '(.*)'},
    'class_name': 'InferredAssetFilesystemDataConnector',
    'module_name': 'great_expectations.datasource.data_connector',
    'base_directory': '/home/pliu/data_set/insee_income'},
   'default_runtime_data_connector_name': {'batch_identifiers': ['default_identifier_name'],
    'class_name': 'RuntimeDataConnector',
    'module_name': 'great_expectations.datasource.data_connector'}},
  'name': 'french_income'}]

# create validation rules

In [10]:
from great_expectations.core.batch import BatchRequest

target_file_name="base_etablissement_par_tranche_effectif.csv"
# create a batch by using data source
batch_request = {
    "datasource_name": datasource_name,
    "data_connector_name": "default_inferred_data_connector_name",
    "data_asset_name": target_file_name,
    "limit": 1000,
}

# give a name of validation rule sets (expectation_suite)
expectation_suite_name = "base_department_validation.rule"

# create an empty expectation suite
suite = data_context.create_expectation_suite(
    expectation_suite_name, overwrite_existing=True # Configure these parameters for your needs
)
# validator groups data source with a validation rule sets (similar to checkpoint)
validator = data_context.get_validator(
    batch_request=BatchRequest(**batch_request),
    expectation_suite_name=expectation_suite_name,
)

# print(type(validator))
# show data source information
column_names = [f'"{column_name}"' for column_name in validator.columns()]
print(f"Columns: {', '.join(column_names)}.")
validator.head(n_rows=5, fetch_all=False)

Calculating Metrics:   0%|          | 0/2 [00:00<?, ?it/s]

Columns: "CODGEO", "LIBGEO", "REG", "DEP", "E14TST", "E14TS0ND", "E14TS1", "E14TS6", "E14TS10", "E14TS20", "E14TS50", "E14TS100", "E14TS200", "E14TS500".


Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

,CODGEO,LIBGEO,REG,DEP,E14TST,E14TS0ND,E14TS1,E14TS6,E14TS10,E14TS20,E14TS50,E14TS100,E14TS200,E14TS500
0,01001,L'Abergement-Clémenciat,82,01,25,22,1,2,0,0,0,0,0,0
1,01002,L'Abergement-de-Varey,82,01,10,9,1,0,0,0,0,0,0,0
2,01004,Ambérieu-en-Bugey,82,01,996,577,272,63,46,24,9,3,2,0
3,01005,Ambérieux-en-Dombes,82,01,99,73,20,3,1,2,0,0,0,0
4,01006,Ambléon,82,01,4,4,0,0,0,0,0,0,0,0


In [13]:
# if you want to ignore certain column, add the column name in the below list
ignored_columns = []

In [14]:
# create a profile,
profiler = UserConfigurableProfiler(
    profile_dataset=validator,
    excluded_expectations=None,
    ignored_columns=ignored_columns,
    not_null_only=False,
    primary_or_compound_key=False,
    semantic_types_dict=None,
    table_expectations_only=False,
    value_set_threshold="MANY",
)

# use the profiler to generate a validation rule set
suite = profiler.build_suite()

Calculating Metrics:   0%|          | 0/2 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/4 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/7 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

...

Calculating Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/7 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/7 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

Creating an expectation suite with the following expectations:

Table-Level Expectations
expect_table_columns_to_match_ordered_list
expect_table_row_count_to_be_between

Expectations by Column
Column Name: CODGEO | Column Data Type: STRING | Cardinality: UNIQUE
expect_column_proportion_of_unique_values_to_be_between
expect_column_values_to_be_in_type_list
expect_column_values_to_not_be_null


Column Name: DEP | Column Data Type: STRING | Cardinality: MANY
expect_column_proportion_of_unique_values_to_be_between
expect_column_values_to_be_in_set
expect_column_values_to_be_in_type_list
expect_column_values_to_not_be_null


Column Name: E14TS0ND | Column Data Type: INT | Cardinality: MANY
expect_column_max_to_be_between
expect_column_mean_to_be_between
expect_column_median_to_be_between
expect_column_min_to_be_between
expect_column_proportion_of_unique_values_to_be_between
expect_column_quantile_values_to_be_between
expect_column_values_to_be_in_set
expect_column_values_to_be_in_type_list


In [15]:
# save the validation rule set to project context
validator.save_expectation_suite(discard_failed_expectations=False)

In [18]:
from great_expectations.checkpoint import SimpleCheckpoint

# create a check point
# Use yaml to configure a checkpoint
auto_gen_ck_name = "checkpoint_for_industry"

ck_config = f"""
name: {auto_gen_ck_name}
config_version: 1.0
class_name: SimpleCheckpoint
run_name_template: "%Y%m%d-%H%M%S-my-run-name-template"
validations:
  - batch_request:
      datasource_name: {datasource_name}
      data_connector_name: default_inferred_data_connector_name
      data_asset_name: {target_file_name}
      data_connector_query:
        index: -1
    expectation_suite_name: {expectation_suite_name}
"""

# preview the checkpoint config
print(ck_config)

#
checkpoint = data_context.test_yaml_config(yaml_config=ck_config)



name: checkpoint_for_industry
config_version: 1.0
class_name: SimpleCheckpoint
run_name_template: "%Y%m%d-%H%M%S-my-run-name-template"
validations:
  - batch_request:
      datasource_name: french_income
      data_connector_name: default_inferred_data_connector_name
      data_asset_name: base_etablissement_par_tranche_effectif.csv
      data_connector_query:
        index: -1
    expectation_suite_name: base_department_validation.rule

Attempting to instantiate class from config...
	Instantiating as a SimpleCheckpoint, since class_name is SimpleCheckpoint
	Successfully instantiated SimpleCheckpoint


Checkpoint class name: SimpleCheckpoint


In [22]:
data_context.add_checkpoint(**yaml.load(ck_config))

{
  "action_list": [
    {
      "name": "store_validation_result",
      "action": {
        "class_name": "StoreValidationResultAction"
      }
    },
    {
      "name": "store_evaluation_params",
      "action": {
        "class_name": "StoreEvaluationParametersAction"
      }
    },
    {
      "name": "update_data_docs",
      "action": {
        "class_name": "UpdateDataDocsAction",
        "site_names": []
      }
    }
  ],
  "batch_request": {},
  "class_name": "Checkpoint",
  "config_version": 1.0,
  "evaluation_parameters": {},
  "module_name": "great_expectations.checkpoint",
  "name": "checkpoint_for_industry",
  "profilers": [],
  "run_name_template": "%Y%m%d-%H%M%S-my-run-name-template",
  "runtime_configuration": {},
  "validations": [
    {
      "batch_request": {
        "datasource_name": "french_income",
        "data_connector_name": "default_inferred_data_connector_name",
        "data_asset_name": "base_etablissement_par_tranche_effectif.csv",
        "data_con

In [24]:
data_context.run_checkpoint(checkpoint_name=auto_gen_ck_name)
data_context.open_data_docs()

Calculating Metrics:   0%|          | 0/173 [00:00<?, ?it/s]